In [8]:
!pip install -q sentence-transformers xgboost pandas scikit-learn pyarrow transformers google-cloud-bigquery

import os
import pandas as pd
import numpy as np


In [9]:
data_dir = "/content"
for root, dirs, files in os.walk(data_dir):
    for f in files:
        if f.endswith(".csv"):
            print(os.path.join(root, f))


/content/patient_risk_predictions.csv
/content/sample_data/encounters.csv
/content/sample_data/observations.csv
/content/sample_data/patients.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/california_housing_train.csv


In [20]:
data_dir = "/content"

patients = pd.read_csv(f"{data_dir}/patients.csv")
encounters = pd.read_csv(f"{data_dir}/encounters.csv")
observations = pd.read_csv(f"{data_dir}/observations.csv")

print("Patients shape:", patients.shape)
print("Encounters shape:", encounters.shape)
print("Observations shape:", observations.shape)


Patients shape: (1163, 25)
Encounters shape: (61459, 15)
Observations shape: (531144, 9)


In [21]:
print("Patients columns:", patients.columns.tolist())
print("Encounters columns:", encounters.columns.tolist())
print("Observations columns:", observations.columns.tolist())


Patients columns: ['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']
Encounters columns: ['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE', 'REASONDESCRIPTION']
Observations columns: ['DATE', 'PATIENT', 'ENCOUNTER', 'CATEGORY', 'CODE', 'DESCRIPTION', 'VALUE', 'UNITS', 'TYPE']


In [29]:
import numpy as np
import re
import pandas as pd
import os

data_dir = "/content"

def try_read(name):
    path = os.path.join(data_dir, name)
    if os.path.exists(path):
        df = pd.read_csv(path, low_memory=False)
        if df.empty:
            return None
        return df
    return None

# explicitly check each option
patients = try_read("patients.csv")
if patients is None:
    patients = try_read("Patients.csv")

encounters = try_read("encounters.csv")
if encounters is None:
    encounters = try_read("Encounters.csv")

observations = try_read("observations.csv")
if observations is None:
    observations = try_read("Observations.csv")

conditions = try_read("conditions.csv")
if conditions is None:
    conditions = try_read("Conditions.csv")

print("Loaded shapes:")
print("patients:", None if patients is None else patients.shape)
print("encounters:", None if encounters is None else encounters.shape)
print("observations:", None if observations is None else observations.shape)
print("conditions:", None if conditions is None else conditions.shape)



# --- Step 1: Process encounters ---
if encounters is not None and not encounters.empty:
    # create a note column
    text_cols = [c for c in encounters.columns if "DESCRIPTION" in c.upper() or "REASON" in c.upper()]
    if text_cols:
        encounters["note"] = encounters[text_cols].fillna("").astype(str).agg(" ".join, axis=1)
    else:
        encounters["note"] = "No description"

    # ICU label: inpatient + "ICU" in description
    encounters["icu_admit"] = encounters.apply(
        lambda x: 1 if (str(x.get("ENCOUNTERCLASS","")).lower()=="inpatient" and "icu" in str(x.get("DESCRIPTION","")).lower()) else 0,
        axis=1
    )

    # patient_id column
    pid_col = None
    for c in ["PATIENT","patient","patient_id","Id","id"]:
        if c in encounters.columns:
            pid_col = c
            break
    if pid_col is None:
        raise SystemExit("Cannot find patient ID in encounters")

    enc_group = encounters.groupby(pid_col).agg({
        "note": lambda x: " ".join(x.astype(str).head(5)),
        "icu_admit": "max"
    }).reset_index().rename(columns={pid_col:"patient_id"})
else:
    enc_group = None

# --- Step 2: Process observations ---
if observations is not None and not observations.empty:
    # detect patient column
    obs_pid_col = None
    for c in ["PATIENT","patient","patient_id","Id","id"]:
        if c in observations.columns:
            obs_pid_col = c
            break
    if obs_pid_col is None:
        raise SystemExit("Cannot find patient ID in observations")

    labs_of_interest = ["WBC","Hemoglobin","Creatinine"]
    obs_group = pd.DataFrame()
    obs_group[obs_pid_col] = observations[obs_pid_col].unique()
    obs_group = obs_group.rename(columns={obs_pid_col:"patient_id"})

    for lab in labs_of_interest:
        matches = observations[observations['DESCRIPTION']==lab]
        if not matches.empty:
            # force numeric conversion
            matches['VALUE_NUM'] = pd.to_numeric(matches['VALUE'], errors='coerce')
            lab_df = matches.groupby(obs_pid_col)['VALUE_NUM'].first().reset_index().rename(columns={obs_pid_col:"patient_id", 'VALUE_NUM':lab})
            obs_group = obs_group.merge(lab_df, on="patient_id", how="left")
        else:
            obs_group[lab] = np.nan

    # fill missing labs with median (numeric only)
    for lab in labs_of_interest:
        obs_group[lab] = pd.to_numeric(obs_group[lab], errors='coerce')
        obs_group[lab] = obs_group[lab].fillna(obs_group[lab].median())
else:
    obs_group = None

# --- Step 3: Process conditions (optional) ---
if conditions is not None and not conditions.empty:
    cond_pid_col = None
    for c in ["PATIENT","patient","patient_id","Id","id"]:
        if c in conditions.columns:
            cond_pid_col = c
            break
    if cond_pid_col is None:
        cond_group = None
    else:
        text_cols = [c for c in conditions.columns if "DESCRIPTION" in c.upper() or "CODE" in c.upper()]
        if text_cols:
            conditions["cond_text"] = conditions[text_cols].fillna("").astype(str).agg(" ".join, axis=1)
        else:
            conditions["cond_text"] = ""
        cond_group = conditions.groupby(cond_pid_col)['cond_text'].apply(lambda x: " ".join(x.astype(str).head(5))).reset_index().rename(columns={cond_pid_col:"patient_id","cond_text":"conditions_text"})
else:
    cond_group = None

# --- Step 4: Merge all per-patient features ---
# start from patient list
if patients is not None and not patients.empty:
    pid_col = None
    for c in ["ID","Id","id","patient_id","PATIENT"]:
        if c in patients.columns:
            pid_col = c
            break
    if pid_col is None:
        pid_col = patients.columns[0]
    df = patients[[pid_col]].rename(columns={pid_col:"patient_id"}).drop_duplicates()
else:
    # fallback: combine enc_group and obs_group
    pid_list = []
    if enc_group is not None:
        pid_list.extend(enc_group['patient_id'].unique())
    if obs_group is not None:
        pid_list.extend(obs_group['patient_id'].unique())
    df = pd.DataFrame({"patient_id": list(set(pid_list))})

# merge in encounters
if enc_group is not None:
    df = df.merge(enc_group, on="patient_id", how="left")

# merge in conditions
if cond_group is not None:
    df = df.merge(cond_group, on="patient_id", how="left")
else:
    df['conditions_text'] = ""

# merge in labs
if obs_group is not None:
    df = df.merge(obs_group, on="patient_id", how="left")
else:
    for lab in ["WBC","Hemoglobin","Creatinine"]:
        df[lab] = np.nan

# --- Step 5: Combined text ---
def safe_join(*parts):
    return ". ".join([str(p) for p in parts if pd.notna(p) and str(p).strip()!=""])

df['combined_text'] = df.apply(lambda r: safe_join(r.get('note',''), r.get('conditions_text','')), axis=1)

print("Final per-patient DataFrame shape:", df.shape)
df.head(3)



Loaded shapes:
patients: (1163, 25)
encounters: (61459, 15)
observations: (531144, 9)
conditions: None
Final per-patient DataFrame shape: (1163, 8)


/tmp/ipython-input-848161018.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['VALUE_NUM'] = pd.to_numeric(matches['VALUE'], errors='coerce')
/tmp/ipython-input-848161018.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['VALUE_NUM'] = pd.to_numeric(matches['VALUE'], errors='coerce')


,patient_id,note,icu_admit,conditions_text,WBC,Hemoglobin,Creatinine,combined_text
0,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,Well child visit (procedure) Well child visi...,0,,NaN,15.25,1.2,Well child visit (procedure) Well child visi...
1,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,Well child visit (procedure) Encounter for s...,0,,NaN,15.25,1.2,Well child visit (procedure) Encounter for s...
2,339144f8-50e1-633e-a013-f361391c4cff,Well child visit (procedure) Emergency room ...,0,,NaN,15.25,1.2,Well child visit (procedure) Emergency room ...


In [32]:
# ------------------------------
# Cell 5: XGBoost + Embeddings + Export (corrected)
# ------------------------------
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sentence_transformers import SentenceTransformer
import pandas as pd
import json

# --- Step 1: Features ---
feature_cols = ['WBC','Hemoglobin','Creatinine']

# Fill missing numeric labs with median
for col in feature_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].fillna(df[col].median())

X = df[feature_cols]

# --- Step 2: Create target ---
if df['icu_admit'].sum() == 0:
    print("No ICU admissions found, generating demo target")
    df['icu_admit_demo'] = ((df['WBC'] > 12) | (df['Creatinine'] > 2)).astype(int)
    y = df['icu_admit_demo'].values
else:
    y = df['icu_admit'].values

# --- Step 3: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 4: Train XGBoost ---
clf = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("XGBoost Accuracy:", acc)

# --- Step 5: Generate Sentence-BERT embeddings ---
model = SentenceTransformer('all-MiniLM-L6-v2')
df['embedding'] = df['combined_text'].fillna("").astype(str).tolist()
embeddings = model.encode(df['embedding'].tolist(), batch_size=32, show_progress_bar=True)
df['embedding'] = embeddings.tolist()

# --- Step 6: Export ---
df.to_csv("/content/patient_risk_predictions.csv", index=False)

with open("/content/patient_embeddings.jsonl","w") as f:
    for idx, row in df.iterrows():
        obj = {
            "patient_id": row['patient_id'],
            "note": row['combined_text'],
            "icu_admit": int(y[idx]),
            "embedding": row['embedding']
        }
        f.write(json.dumps(obj) + "\n")

print("✅ Files ready: patient_embeddings.jsonl, patient_risk_predictions.csv")


No ICU admissions found, generating demo target
XGBoost Accuracy: 1.0


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [16:23:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

✅ Files ready: patient_embeddings.jsonl, patient_risk_predictions.csv
